In [1]:
import neo4j

import csv
import os 
import math
import numpy as np
import pandas as pd
from geographiclib.geodesic import Geodesic

import psycopg2

## Neo4j & PostgreSQL Supporting Code (provided from the labs)

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

In [6]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [7]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [8]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [9]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [10]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [11]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [12]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [13]:
cursor = connection.cursor()

# Dropping Tables

In [14]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()


In [15]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor.execute(query)

connection.commit()


In [16]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


# Create Data Tables

In [17]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

In [18]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

In [19]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

# Load Data into Tables

In [20]:
connection.rollback()

query = """

copy stations
from '/user/projects/project-3-nicholasllin/code/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [21]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-nicholasllin/code/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [22]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-nicholasllin/code/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

# Wipe Neo4j

In [23]:
my_neo4j_wipe_out_database()

In [24]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Load Stations & Relationships

In [25]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [26]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [27]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [28]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [29]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [30]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [31]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [32]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# Running Louvains Algorithm To Identify Clusters and Potential Pickup Locations

In [33]:
#Setup Neo4j DS Algorithm Graphing
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = """

CALL gds.graph.project('ds_graph', 'Station', 'LINK', 
                      {relationshipProperties: 'weight'})

"""
session.run(query)

In [34]:
#Run Louvain on Graph
query = """

CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

louvain_df = my_neo4j_run_query_pandas(query)
louvain_df.head()

,name,community,intermediate_community
0,arrive 12th Street,41,"[41, 41, 41]"
1,arrive 19th Street,41,"[45, 41, 41]"
2,arrive MacArthur,41,"[93, 41, 41]"
3,depart 12th Street,41,"[41, 41, 41]"
4,depart 19th Street,41,"[45, 41, 41]"


In [35]:
#reindex cluster list
community_list_old = sorted(list(louvain_df['community'].unique()))
print(community_list_old)
community_list_new = [i for i in range(len(community_list_old))]
print(community_list_new)

[41, 47, 55, 69, 73, 75, 79, 91, 97, 115, 129, 133]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [36]:
#apply reindexed cluster list
louvain_df['community'] = louvain_df['community'].replace(community_list_old, community_list_new)
louvain_df = louvain_df[['name', 'community']]

In [37]:
#remove arrive / depart & colors on station names
def clean_louvain(x):
    x_list = x.split(' ')
    x_list.pop(0)
    station = ' '.join(x_list)
    return station

louvain_df['name'] = louvain_df['name'].apply(clean_louvain)
louvain_df = louvain_df.drop_duplicates()

In [38]:
louvain_df

,name,community
0,12th Street,0
1,19th Street,0
2,MacArthur,0
15,16th Street Mission,1
16,24th Street Mission,1
27,Bay Fair,2
28,Coliseum,2
29,OAK,2
30,San Leandro,2
46,Balboa Park,3


# Reading Ridership Data

In [39]:
#ADD 3 MONTH RIDERSHIP DATA
data_path = '/user/projects/project-3-nicholasllin/code/ridership_2024'

In [40]:
#load in all data
total_ridership = pd.DataFrame()
for filename in os.listdir(data_path):
    f = os.path.join(data_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        temp_ridership = pd.read_excel(f, header = 4, sheet_name = 0)
        temp_ridership = temp_ridership.rename(columns = {'Exit Station Two-Letter Code':'Exit Stations','Unnamed: 51':'Exit Counts'})
        temp_ridership = temp_ridership[temp_ridership['Exit Stations'] != 'Grand Total']
        temp_ridership = temp_ridership[['Exit Stations', 'Exit Counts']]
    total_ridership = pd.concat([total_ridership, temp_ridership])
total_ridership

,Exit Stations,Exit Counts
0,RM,52671.0
1,EN,86380.0
2,EP,49372.0
3,NB,41330.0
4,BK,124864.0
...,...,...
45,WS,28614.0
46,PC,10963.0
47,AN,40337.0
48,ML,29799.0


In [41]:
#group by stations to sum number of exits
total_ridership = total_ridership.groupby(['Exit Stations'])['Exit Counts'].sum().reset_index()
total_ridership.head()

,Exit Stations,Exit Counts
0,12,351839.0
1,16,432192.0
2,19,326269.0
3,24,416375.0
4,AN,113360.0


# Reading in Station Abreviation Data

In [42]:
station_abv = pd.read_excel('/user/projects/project-3-nicholasllin/code/station-names.xls')
station_abv = station_abv.drop(columns = ['Unnamed: 0'])
station_abv['Two-Letter Station Code'] = station_abv['Two-Letter Station Code'].astype(str)

In [43]:
#Convert Abreviated Stations to Full Stations
station_abv_dict = station_abv.set_index(str('Two-Letter Station Code'))['Station Name'].to_dict()
def convert_stations(x):
    y = station_abv_dict[x]
    return y

total_ridership['Exit Stations'] = total_ridership['Exit Stations'].apply(convert_stations)
total_ridership.head()

,Exit Stations,Exit Counts
0,12th Street / Oakland City Center,351839.0
1,16th Street Mission,432192.0
2,19th Street Oakland,326269.0
3,24th Street Mission,416375.0
4,Antioch,113360.0


# Merge tables together to identify most popular exits per community

In [44]:
merged = louvain_df.merge(total_ridership, left_on = 'name', right_on = 'Exit Stations', how = 'inner')
merged = merged.drop_duplicates()
merged = merged[['name', 'community', 'Exit Counts']]
merged = merged.sort_values(by = 'Exit Counts', ascending = False)
merged.head()

,name,community,Exit Counts
13,Embarcadero,6,1142339.0
24,Montgomery Street,9,972388.0
25,Powell Street,9,852125.0
23,Civic Center,9,641412.0
1,16th Street Mission,1,432192.0


In [45]:
#looking at all the stations, community, and exit values
merged = merged.sort_values(by = 'community', ascending = False)
merged

,name,community,Exit Counts
35,Pittsburg Center,11,32492.0
36,Pleasant Hill,11,164164.0
31,Concord,11,176233.0
33,North Concord,11,41211.0
37,Rockridge,11,183112.0
38,Walnut Creek,11,194975.0
34,Orinda,11,85055.0
30,Antioch,11,113360.0
32,Lafayette,11,109913.0
28,San Bruno,10,108010.0


In [58]:
#looking at top exits per community
top_exits = merged.groupby(['community'])[['name','Exit Counts']].max().reset_index()
top_exits['Exit Counts'] = top_exits['Exit Counts'] // 3
top_exits = top_exits.rename(columns={'community':'Cluster','name':'Station',
                                     'Exit Counts':'Avg Total Exits Over 3 Months'})
top_exits

,Cluster,Station,Avg Total Exits Over 3 Months
0,0,MacArthur,88943.0
1,1,24th Street Mission,144064.0
2,2,San Leandro,76013.0
3,3,Glen Park,111682.0
4,4,Castro Valley,27956.0
5,5,Richmond,52704.0
6,6,West Oakland,380779.0
7,7,Lake Merritt,92132.0
8,8,Warm Springs,51099.0
9,9,Powell Street,324129.0


# Running PageRank to Identify Potential Pickup Locations

In [47]:
# Find the Page Rank for each stop


query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)
query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

query = """

CALL gds.pageRank.stream('ds_graph',
                         { maxIterations: $max_iterations,
                           dampingFactor: $damping_factor}
                         )
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as page_rank
ORDER BY page_rank DESC, name ASC

"""

max_iterations = 100
damping_factor = 0.85

results = my_neo4j_run_query_pandas(query, max_iterations=max_iterations, damping_factor=damping_factor).head(20)
print(results)

                       name  page_rank
0             blue Coliseum   0.690175
1           orange Coliseum   0.689562
2            green Coliseum   0.689359
3             blue Bay Fair   0.675909
4          yellow MacArthur   0.675697
5             red Daly City   0.674432
6          yellow Daly City   0.674333
7   yellow Pittsburg Center   0.657570
8          yellow Pittsburg   0.645705
9       yellow West Oakland   0.645436
10         red West Oakland   0.645319
11        blue West Oakland   0.644530
12       green West Oakland   0.644527
13     yellow North Concord   0.641978
14         orange MacArthur   0.640724
15           yellow Concord   0.640689
16            red MacArthur   0.640541
17     yellow Pleasant Hill   0.639865
18          orange Bay Fair   0.639564
19           green Bay Fair   0.639560


In [48]:
# Remove the line name for each stop
def remove_line():
    station_only = results.head(20)
    for idx in station_only.index:
        row = station_only['name'][idx]
        station_only.loc[idx, 'name'] = row.partition(' ')[2]
    return station_only    

station_only = remove_line()

In [55]:
# Group by and find the average PageRank
def average_page_rank():
    avg_rank = station_only.groupby('name')['page_rank'].mean().reset_index()
    avg_rank.columns = ["Station", "Average PageRank"]
    return avg_rank.sort_values(by=['Average PageRank'], ascending=False).reset_index(drop=True)

average_page_rank()

,Station,Average PageRank
0,Coliseum,0.689699
1,Daly City,0.674382
2,Pittsburg Center,0.657570
3,MacArthur,0.652321
4,Bay Fair,0.651678
5,Pittsburg,0.645705
6,West Oakland,0.644953
7,North Concord,0.641978
8,Concord,0.640689
9,Pleasant Hill,0.639865


# Shortest Paths for Customer Pickup

The purpose of this use case is to identify the shortest BART path a customer should take from their home to our BART pickup locations. For this POC, we will be using the customers in the stage_1_peak_customers table. There are several steps involved:
1. Identify each customer's closest BART station: use geodesic distance calculation to identify the closest station based on the customer's zip code.
    1. Collect distinct zip codes from the stage_1_peak_customers table
    2. Use the zip_codes table to get the longitude and latitude for each zip code
    3. Find the closest station to that zip code through geodesic distance calculation, store zip codes and stations in a dictionary.
    4. Use the dictionary to add the closest station to each customer, based on their zip code
2. Run Dijkstra's shortest path algorithm between each customer's station and the pickup stations.
    1. Identify our ideal BART pickup stations (via use case #1 that other teammates are working on)
    2. Run dijkstra's shortest path algorithm between the customer's station and each pickup station.
    3. Compare the cost of travelling to each pickup location, and determine which station is most ideal for us to drop their food at. 

### 1. Identify each customer's closest BART station

In [50]:
# Provided function
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [51]:
def my_zip_get_stations(zipcode, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from zip_codes "
    query += "where zip = '" + zipcode + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select station from stations "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    if len(rows) > 0:        
        return rows[0][0]
    
    else:
        return False

Discussed with Doris: we are leveraging the stage_1_peak_customers table that we previously created as a part of Project 2. These customers will serve as a POC for our business case. Doris mentioned it is ok to assume the reader already has this table stored in their database from the previous project.

In [52]:
# Find the closest station for each zip code in the customers table, and store in a dict
query = """
    select distinct zip
    from stage_1_peak_customers
"""

distinct_zips = pd.read_sql_query(query, connection)

station_dict = {}

for zipcode in distinct_zips['zip']:
    distance = 0.25
    station = False
    while station == False:
        station = my_zip_get_stations(zipcode,distance)
        distance += 0.01
    
    station_dict[zipcode] = station

# Store the customers table in a dataframe, and append each customer's closest station
query = """
select *
from stage_1_peak_customers
"""

customer_df = pd.read_sql_query(query, connection)

customer_df['closest_station'] = customer_df['zip'].map(station_dict)
customer_df

,stage_id,sale_id,customer_id,first_name,last_name,street,city,state,zip,closest_station
0,1,5763728874,3728404,Darrelle,Dohrmann,46 Farwell Terrace,Oakland,CA,94609,MacArthur
1,2,5763729036,3729309,Moria,Greenwood,8803 Delaware Crossing,Berkeley,CA,94705,Rockridge
2,3,5763728904,3728508,Josiah,Hulett,6755 Melby Plaza,Oakland,CA,94612,19th Street
3,4,5763728973,3728534,Gayle,MacGarrity,286 Onsgard Center,Berkeley,CA,94703,Downtown Berkeley
4,5,5763728757,3729188,Courtenay,Shirrell,75 West Park,Emeryville,CA,94608,MacArthur
...,...,...,...,...,...,...,...,...,...,...
92,93,5763728927,3728568,Valina,Spring,119 Sachtjen Junction,Berkeley,CA,94702,North Berkeley
93,94,5763729096,3728990,Claire,Mebes,358 Oxford Road,Albany,CA,94706,El Cerrito Plaza
94,95,5763729268,3728901,Freddy,Mumford,6 Transport Lane,Orinda,CA,94563,Orinda
95,96,5763729237,3729019,Arv,Doret,2120 Mesta Circle,Emeryville,CA,94608,MacArthur


### 2. Run Dijkstra's shortest path algorithm

In [53]:
def my_neo4j_shortest_path_minutes(from_station, to_station, print_nodes=False):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        total_cost = int(r['totalCost'])
        minutes = round(total_cost / 60.0,1)
        
        if print_nodes:
            print("\n--------------------------------")
            print("   Total Cost: ", total_cost)
            print("   Minutes: ", minutes)
            print("--------------------------------")

            nodes = r['nodes']
            costs = r['costs']

            i = 0
            previous = 0

            for n in nodes:

                print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))

                previous = int(costs[i])
                i += 1
            
            return None

        return total_cost, minutes

In [59]:
# For now, let's say our 6 pickup spots are 'Coliseum', '24th Street Mission', 'West Oakland', 'Warm Springs', 'Richmond', 'Walnut Creek'
# Our function will compare the time for each customer to get to these stops, and return the most ideal one

pickup_stations = ['Coliseum', '24th Street Mission', 'West Oakland', 'Warm Springs', 'Richmond', 'Walnut Creek'] 
customer_station = 'MacArthur' #one example

cols = ['customer_station','pickup_station','total_cost','minutes']
path_costs = pd.DataFrame(columns=cols)

for pickup_station in pickup_stations:
    total_cost, minutes = my_neo4j_shortest_path_minutes('depart ' + customer_station, 'arrive ' + pickup_station)
    path = pd.DataFrame(columns=cols, data=[[customer_station, pickup_station, total_cost, minutes]])
    path_costs = pd.concat([path_costs, path])
    
path_costs = path_costs.sort_values(['minutes', 'total_cost'],ascending=[True, True]).reset_index(drop=True)
ideal_station = path_costs['pickup_station'][0]

print('This customers ideal pickup station is: ',ideal_station)
my_neo4j_shortest_path_minutes('depart ' + customer_station, 'arrive ' + ideal_station, print_nodes=True)

This customers ideal pickup station is:  West Oakland

--------------------------------
   Total Cost:  600
   Minutes:  10.0
--------------------------------
depart MacArthur, 0, 0
yellow MacArthur, 0, 0
yellow 19th Street, 180, 180
yellow 12th Street, 120, 300
yellow West Oakland, 300, 600
arrive West Oakland, 0, 600
